In [7]:
cd ..

/content


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!curl -L 'https://public.roboflow.com/ds/WWHF51u4si?key=vOgM6mvbVh' > roboflow.zip; unzip roboflow.zip

In [1]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14967, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 14967 (delta 13), reused 16 (delta 6), pack-reused 14936
Receiving objects: 100% (14967/14967), 13.95 MiB | 18.39 MiB/s, done.
Resolving deltas: 100% (10277/10277), done.


In [7]:
ls

benchmarks.py    data/       LICENSE          requirements.txt  tutorial.ipynb
CITATION.cff     detect.py   models/          segment/          utils/
classify/        export.py   README.md        setup.cfg         val.py
CONTRIBUTING.md  hubconf.py  README.zh-CN.md  train.py


In [3]:
cd yolov5/

/content/yolov5


In [4]:
## 필요 라이브러리 설치
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.4 MB/s eta 0:00:00


In [10]:
from glob import glob

In [11]:
img_list=glob('/content/export/images/*jpg')

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
## data.yaml file에 트레인과 테스트로 나눈 경로를 명시
## nc는 class count
train_img_list , test_img_list = train_test_split(img_list, test_size=0.2, random_state=0)

In [14]:
## 파일 만들기 , \n -> 한줄씩 들어가게
with open('/content/dataset/train.txt', 'w') as f :
  f.write('\n'.join(train_img_list)+'\n')

with open('/content/dataset/test.txt', 'w') as f :
  f.write('\n'.join(test_img_list)+'\n')

In [18]:
cd yolov5/

/content/yolov5


In [ ]:
## train.py 파일 실행
## --data ../data.yaml 이 yaml 파일엔 데이터가 들어 있는 경로 명시 // --freeze 10 비활성 conv layers num
!python train.py --img 416 --batch 16 --epochs 50 --data ../data.yaml --cfg ./models/yolov5s.yaml --freeze 10

In [ ]:
## 텐서보드로 결과값 출력 logdir 다음에 경로지정
%load_ext tensorboard
%tensorboard --logdir '/content/yolov5/runs'

In [21]:
# 해당 모델로 학습한 예측 데이터전체 확인
!python val.py --data ../data.yaml --weights /content/yolov5/runs/train/exp/weights/best.pt

val: data=../data.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-63-gcdd804d Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/dataset/test.cache... 595 images, 0 backgrounds, 0 corrupt: 100% 595/595 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 19/19 [00:07<00:00,  2.46it/s]
                   all        595        705      0.502      0.467      0.492      0.206
Speed: 0.2ms pre-process, 6.1ms inference, 1.7ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/val/exp


In [27]:
## image - box detection // teset list[0] 에 대한 detection
!python detect.py --weights '/content/yolov5/runs/train/exp2/weights/best.pt' --img 416 --conf 0.5 --source "{test_img_list[0]}"

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=/content/export/images/armas (2178)_jpg.rf.0095da5f53aa7d641eaeafc9c9b9b9ab.jpg, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-63-gcdd804d Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/export/images/armas (2178)_jpg.rf.0095da5f53aa7d641eaeafc9c9b9b9ab.jpg: 416x416 1 pistol, 8.2ms
Speed: 0.4ms pre-process, 8.2ms inference, 1.6ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp


In [26]:
test_img_list[0]

'/content/export/images/armas (2178)_jpg.rf.0095da5f53aa7d641eaeafc9c9b9b9ab.jpg'

In [31]:
## 영상에 학습된 권총 영역 옮기기 conf == mean average precision 의 임계값 설정
!python detect.py --weights '/content/yolov5/runs/train/exp2/weights/best.pt' --conf 0.6 --source ./pp3.mp4 

detect: weights=['/content/yolov5/runs/train/exp2/weights/best.pt'], source=./pp3.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-63-gcdd804d Python-3.8.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/1185) /content/yolov5/pp3.mp4: 640x384 (no detections), 13.1ms
video 1/1 (2/1185) /content/yolov5/pp3.mp4: 640x384 (no detections), 9.4ms
video 1/1 (3/1185) /content/yolov5/pp3.mp4: 640x384 (no detections), 9.4ms
video 1/1 (4/1185) /content/yolov5/pp3.mp4: 640x384 (no detections), 9.4ms
video 1/1 (5/1185) /content/yolov5/pp3